In [ ]:
import pandas as pd
import 

In [ ]:
# your file list
import os
os.chdir('///') ### change this! 
files = os.listdir('.')
files

In [ ]:
for file in files:
    data = pd.read_csv(file)
    print(file)
    print(data.shape)
    
    filename = file.split('.')[0]+'_pron.csv'
    pron = pronoun_analysis(file, filename)
    #print('after prons: '+ str(pron.shape))
    #print(filename)
    
    filename = file.split('.')[0]+'_time.csv'
    time = time_analysis(file, filename)
    #print('after tine: '+ str(time.shape))
    #print(filename)
    
    print('--------')

In [ ]:
def make_lists(filename):
    df = pd.read_csv(filename)
    
    content_lists = []
    #for i in (df['title'] + ' '+df['text_context']):
    for i in df['post']:
        i = i.replace('\n', '')
        content_lists.append(i.lower())
    return content_lists

## Linguistic Features & Sentimental Analysis

- LIWCalike

```r
library("quanteda.dictionaries")
library("corpus")


file = read.csv("/Users/midan/Downloads/final/final_all_4.csv")  # here you should change yours
file[2,7] # title + text

contents = c(file[1,7])
for (i in 2:nrow(file)){
  temp = paste(file[i,7]) 
  contents = append(contents,temp, after = i-1)
}


nrow(file)
length(output_lsd)
length(contents)
output_lsd <- liwcalike(contents, dictionary = data_dictionary_NRC)
write.csv(output_lsd, '/Users/midan/Downloads/final/final_all_4_LIWCalike.csv')  # here you should change yours

```

## Pronouns analysis & Time oriented analysis

In [ ]:
## pronons analysis
def pronoun_analysis(filename,filename2):
    content_lists = make_lists(filename)
    pers1 = ['i ', 'my', 'me', 'mine', "i'm", "i'd", "i've", "i'll", 'I ']
    pers1_p = ['we', 'our', 'us', 'ours', "we're", "we'd", "we'll"]
    pers2 = ['you', 'u', 'yours', 'your', 'ur', "you'll", "you've", "you'd"]
    pers3 = ['he', 'she', 'him', 'his', 'hers', 'her', "she's", "she'd", "she'll", "he's", "he'd", "he'll", 
        "she've", "he've"]
    pers3_p = ['them', 'their','they', 'theirs',  "they're", "they'd", "they'll",  "they've"]
    pers0 = ['it', 'that',  'this', 'its', "it's", "that's", "these", "those"]
    
    list1 = []
    list1_p = []
    list2 = []
    list3 = []
    list3_p = []
    list0 = []
    cnt = 0
    for content in content_lists:
        cnt+=1
        if type(content) == float: 
            print(cnt)
            print(content)
            temp = content_lists[cnt]
            wc = len(temp)
            count1 = 0 # first person singular
            count1_p = 0 # first person plural
            count2 = 0 # second person
            count3 = 0 # third person singular
            count3_p = 0 # third person plural
            count4 = 0 # impersonal
            
            for c in temp:
                for pers in pers1: 
                    if pers == c: count1+=1
                for pers in pers2: 
                    if pers == c: count2+=1
                for pers in pers3: 
                    if pers == c: count3+=1
                for pers in pers0: 
                    if pers == c: count4+=1
                for pers in pers1_p: 
                    if pers == c: count1_p+=1
                for pers in per3_p1: 
                    if pers == c: count3_p+=1
        else:
            content = content.split()
            wc = len(content)
            count1 = 0 # first person singular
            count1_p = 0 # first person plural
            count2 = 0 # second person
            count3 = 0 # third person singular
            count3_p = 0 # third person plural
            count4 = 0 # impersonal
            for c in content:
                for pers in pers1: 
                    if pers == c: count1+=1
                for pers in pers2: 
                    if pers == c: count2+=1
                for pers in pers3: 
                    if pers == c: count3+=1
                for pers in pers0: 
                    if pers == c: count4+=1
                for pers in pers1_p: 
                    if pers == c: count1_p+=1
                for pers in pers3_p: 
                    if pers == c: count3_p+=1
            list1.append(count1/wc)
            list1_p.append(count1_p/wc)
            list2.append(count2/wc)
            list3.append(count3/wc)
            list3_p.append(count3_p/wc)
            list0.append(count4/wc)
    result = pd.DataFrame({
        '1pers': list1, '1pers_plural': list1_p, '2pers': list2, '3pers': list3,'3pers_plural': list3_p, 'nonpers': list0, 
    })
    result.to_csv(filename2, index = False)
    #print('done!')
    return result

In [ ]:
def time_analysis(filename, filename2):   
    content_lists = make_lists(filename)
    past =['ago', 'did', 'before', 'yesterday', 'last']
    present = ['present', 'now', 'today', 'this']
    future = ['future', 'may', 'will', 'soon', 'next', 'gonna']
    vb_pr = ['VBP', 'VBZ', 'VB']
    
    future_s = []
    present_s = []
    past_s = []
    for j in range(len(content_lists)):
        #print(j)
        content = content_lists[j]
        if type(content)==float: f_s, pr_s, p_s = -1, -1, -1
        else:
            temp_text = nltk.word_tokenize(content)
            f_s = 0 # future
            pr_s = 0 # present
            p_s = 0 #past
            for t in temp_text:
                if t in future: f_s +=1
                if t in present: pr_s +=1
                if t in past: p_s +=1
            temp_tag = nltk.pos_tag(temp_text)
            for i in range(len(temp_tag)):
                if temp_tag[i][1] in vb_pr: pr_s +=1
                elif temp_tag[i][1] == 'VBD': p_s +=1
        future_s.append(f_s)
        present_s.append(pr_s)
        past_s.append(p_s)
    result = pd.DataFrame({
       'past':past_s, 'present': present_s ,'future': future_s
    })
    result.to_csv(filename2, index = False)
    return result

# Concat

In [ ]:
import pandas as pd

In [ ]:
import os 
os.chdir('/Users/midan/Downloads/final/')
files = os.listdir()
files

In [ ]:
for i in ['final_all_1', 'final_all_2', 'final_all_3', 'final_all_4', 'final_all_5']:
    df = pd.read_csv(i+'.csv')
    pron = pd.read_csv(i+'_pron.csv')
    time = pd.read_csv(i+'_time.csv')
    liwc = pd.read_csv(i+'_LIWCalike.csv')
    liwc = liwc[['WPS', 'WC', 'Sixltr','anger', 'anticipation', 'disgust', 'fear', 'joy', 'negative',
       'positive', 'sadness', 'surprise', 'trust']]
    result = pd.concat([df, liwc, pron, time])
    print(result.shape)
    result.to_csv('LIWC_'+i+'.csv', index = False)